In [17]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
import numpy as np
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'matplotlib'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'matplotlib'



In [18]:
spark = SparkSession.builder.appName('ml-bank').getOrCreate()

input_bucket = "s3://cdcolganbucket"

input_path = '/eCommerce.csv'

df = spark.read.csv(input_bucket + input_path, header = True, inferSchema = True)

#df = spark.read.csv('bank.csv', header = True, inferSchema = True)

df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: integer (nullable = true)
 |-- Unnamed: 0: integer (nullable = true)
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

In [19]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                  0  ...                                     4
_c0                                        20304271  ...                               1115695
Unnamed: 0                                 32411428  ...                               1783756
event_time                  2019-10-24 08:20:17 UTC  ...               2019-10-02 10:44:02 UTC
event_type                                     view  ...                                  view
product_id                                  1004785  ...                               5100737
category_id                     2053013555631882655  ...                   2053013553341792533
category_code                electronics.smartphone  ...                    electronics.clocks
brand                                        huawei  ...                               samsung
price                                        270.42  ...                                150.95
user_id                                   55893591

In [20]:
df.groupby('event_type').count().toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  event_type    count
0       cart   266914
1       view  8316245
2   purchase   181846

In [21]:
#pd.DataFrame(df[df["event_type"] == "cart"].take(5), columns=df[df["event_type"] == "cart"].columns).transpose()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().toPandas().transpose()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  0                     1  ...          3          4
summary       count                  mean  ...        min        max
_c0         8765005  1.3283255473634984E7  ...          0   26560615
Unnamed: 0  8765005   2.127074151289623E7  ...          1   42448757
product_id  8765005     5366080.760621015  ...    1000978   60500010
user_id     8765005   5.339297623416629E8  ...  195082191  566280676

[5 rows x 5 columns]

In [23]:
numeric_data = df.select(numeric_features).toPandas()
plt.clf()
axs = scatter_matrix(numeric_data, figsize=(7, 7));
# Rotate axis labels and remove axis ticks
n = len(numeric_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())
%matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'plt' is not defined
Traceback (most recent call last):
NameError: name 'plt' is not defined



In [24]:
df = df.select("category_code", "brand", "price", "event_type")

cols = df.columns

df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- event_type: string (nullable = true)

In [25]:
categoricalColumns = ["category_code", "brand", "price"]

stages = []

for categoricalCol in categoricalColumns:
    
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    
    stages += [stringIndexer, encoder]

label_stringIdx = StringIndexer(inputCol = 'event_type', outputCol = 'label')

stages += [label_stringIdx]

numericCols = []

assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols

assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

stages += [assembler]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
pipeline = Pipeline(stages = stages)

pipelineModel = pipeline.fit(df)

df = pipelineModel.transform(df)

selectedCols = ['label', 'features'] + cols

df = df.select(selectedCols)

df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- event_type: string (nullable = true)

In [27]:
train, test = df.randomSplit([0.7, 0.3], seed = 2018)

print("Training Dataset Count: " + str(train.count()))

print("Test Dataset Count: " + str(test.count()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training Dataset Count: 6136185
Test Dataset Count: 2628820

In [28]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)

lrModel = lr.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
beta = np.sort(lrModel.coefficients)

plt.clf()

plt.plot(beta)

plt.ylabel('Beta Coefficients')

plt.show()

%matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o1223.coefficients.
: org.apache.spark.SparkException: Multinomial models contain a matrix of coefficients, use coefficientMatrix instead.
	at org.apache.spark.ml.classification.LogisticRegressionModel.coefficients(LogisticRegression.scala:955)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:74

In [30]:
trainingSummary = lrModel.summary

roc = trainingSummary.roc.toPandas()

plt.clf()

plt.plot(roc['FPR'],roc['TPR'])

plt.ylabel('False Positive Rate')

plt.xlabel('True Positive Rate')

plt.title('ROC Curve')

plt.show()

%matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'LogisticRegressionTrainingSummary' object has no attribute 'roc'
Traceback (most recent call last):
AttributeError: 'LogisticRegressionTrainingSummary' object has no attribute 'roc'



In [31]:
pr = trainingSummary.pr.toPandas()

plt.clf()

plt.plot(pr['recall'],pr['precision'])

plt.ylabel('Precision')

plt.xlabel('Recall')

plt.show()

%matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'LogisticRegressionTrainingSummary' object has no attribute 'pr'
Traceback (most recent call last):
AttributeError: 'LogisticRegressionTrainingSummary' object has no attribute 'pr'



In [32]:
f = trainingSummary.fMeasureByThreshold.toPandas()

plt.clf()

plt.plot(f['threshold'],f['F-Measure'])

plt.ylabel('F-Measure')

plt.xlabel('Threshold')

plt.show()

%matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'LogisticRegressionTrainingSummary' object has no attribute 'fMeasureByThreshold'
Traceback (most recent call last):
AttributeError: 'LogisticRegressionTrainingSummary' object has no attribute 'fMeasureByThreshold'



In [33]:
predictions = lrModel.transform(test)

predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
"cannot resolve '`age`' given input columns: [price, features, prediction, event_type, label, rawPrediction, probability, category_code, brand];;\n'Project ['age, 'job, label#1200, rawPrediction#1362, prediction#1379, probability#1370]\n+- Project [label#1200, features#1212, category_code#693, brand#694, price#695, event_type#690, rawPrediction#1362, probability#1370, UDF(rawPrediction#1362) AS prediction#1379]\n   +- Project [label#1200, features#1212, category_code#693, brand#694, price#695, event_type#690, rawPrediction#1362, UDF(rawPrediction#1362) AS probability#1370]\n      +- Project [label#1200, features#1212, category_code#693, brand#694, price#695, event_type#690, UDF(features#1212) AS rawPrediction#1362]\n         +- Sample 0.7, 1.0, false, 2018\n            +- Sort [label#1200 ASC NULLS FIRST, features#1212 ASC NULLS FIRST, category_code#693 ASC NULLS FIRST, brand#694 ASC NULLS FIRST, price#695 ASC NULLS FIRST, event_type#690 ASC NULLS FIRST], fals

In [34]:
evaluator = BinaryClassificationEvaluator()

print('Test Area Under ROC', evaluator.evaluate(predictions))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Area Under ROC 0.7159453620956935

In [35]:
evaluator.getMetricName()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'areaUnderROC'

In [ ]:
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)

dtModel = dt.fit(train)

predictions = dtModel.transform(test)

predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
evaluator = BinaryClassificationEvaluator()

print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

In [ ]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')

rfModel = rf.fit(train)

predictions = rfModel.transform(test)

predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

In [ ]:
evaluator = BinaryClassificationEvaluator()

print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))